# 第6章 逻辑斯谛回归

配置环境：python 3.6

代码全部测试通过。

代码参考 : https://github.com/fengdu78/lihang-code/blob/master/code/%E7%AC%AC6%E7%AB%A0%20%E9%80%BB%E8%BE%91%E6%96%AF%E8%B0%9B%E5%9B%9E%E5%BD%92(LogisticRegression)/%E6%9C%80%E5%A4%A7%E7%86%B5%E6%A8%A1%E5%9E%8B%20IIS.py

此文档方便阅读，若需要复制粘贴可以在当前目录中查看`MaxEntropy.py`